In [24]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import yaml

import plotly.graph_objects as go


In [37]:
def bins(n):
    bins = []
    for i in range(n):
        bins.append(2**(i+1))
    return bins

def compute_cdf(a):
    x = np.sort(a);
    p = np.array(range(len(a)))/float(len(a)-1)
    return x, p

In [49]:
# Series of scores from my games.
jeff_scores = np.array([30080, 2460, 7396, 8280, 26420, 26304, 9836, 28012, 5260, 16540, 19148, 20440, 15300, 8156, 7096, 28248, 27268, 32184, 25852, 32324, 32768])

jeff_cdf_x, jeff_cdf_y = compute_cdf(jeff_scores)

In [83]:
with open('report.yml') as f:
    data = yaml.load(f.read())
    
# # Cheat and read lookahead 5 from old report (it's slow to run)
# with open('report_1.yml') as f:
#     tmp = yaml.load(f.read())
#     data['naive_lookahead5'] = tmp['naive_lookahead5']

/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [87]:
def sortAlg(alg):
    # Weight by the average of the largest block
    return np.mean(np.array(bins(len(alg['largest_hist']))) * np.array(alg['largest_hist']))

sorted_names = sorted(data.keys(), key=lambda n: sortAlg(data[n]))

plt.figure()
for name in sorted_names:
    d = data[name]
    plt.plot(d['score_cdf_x'], 1-np.array(d['score_cdf_y']), label=name)
plt.plot(jeff_cdf_x, 1-jeff_cdf_y, '-^', label='Jeff')
plt.title('Final Score CDF')
plt.legend()

/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:



bar_values = { k: d['largest_hist'] for k, d in data.items() }
max_size = max([len(x) for x in bar_values.values()])

# Pad any short arrays with zeros, and normalize
for k in bar_values.keys():
    missing = max_size - len(bar_values[k])
    if missing > 0:
        bar_values[k] += [0] * missing
    
    bar_values[k] = np.array(bar_values[k], dtype=np.float)
    bar_values[k] *= 100 / bar_values[k].sum()
    
print(bar_values)

labels = ["g"+str(x) for x in bins(max_size)]

print(labels)

fig = go.FigureWidget(data=[
    go.Bar(name=name, x=labels, y=bar_values[name]) 
    for name in sorted_names
    ])
# Change the bar mode
fig.update_layout(barmode='group')
#fig.show()
# for name, d in data.items():
#     plt.hist(d['largest_hist'], bins(len(d['largest_hist'])), label=name)
    
# plt.legend()

{'lookahead1': array([ 0.,  0.,  0.,  0.,  1.,  4., 40., 44., 11.,  0.,  0.]), 'lookahead3': array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  8. , 53.5, 38. ,  0. ]), 'lookahead5': array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1.5, 37.5, 56. ,  5. ]), 'lookaheadsorted1': array([ 0. ,  0. ,  0. ,  0. ,  0. ,  2. , 14.5, 51.5, 31.5,  0.5,  0. ]), 'lookaheadsorted3': array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  2. , 20.5, 61. , 16.5]), 'lookaheadsorted5': array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 11.5, 52. , 36.5]), 'max_free_space': array([ 0. ,  0. ,  0. ,  0. ,  0. ,  3.5, 34. , 48. , 14.5,  0. ,  0. ]), 'max_free_space_3dir': array([ 0. ,  0. ,  0. ,  0. ,  0. ,  4. , 43. , 40. , 12.5,  0.5,  0. ]), 'random': array([ 0. ,  0. ,  0. ,  1. ,  5. , 33.5, 52.5,  8. ,  0. ,  0. ,  0. ]), 'random_3dir': array([ 0. ,  0. ,  0.5,  0.5,  6.5, 26.5, 44.5, 20. ,  1.5,  0. ,  0. ])}
['g2', 'g4', 'g8', 'g16', 'g32', 'g64', 'g128', 'g256', 'g512', 'g1024', 'g2048']


FigureWidget({
    'data': [{'name': 'random',
              'type': 'bar',
              'uid': '214fb3e5-8e0…